<a href="https://colab.research.google.com/github/nguyenfan20/AI-Agents-using-LangChain/blob/main/VietNam_Local_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install torchvision --upgrade
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence_transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference
    !pip install spacy
    !pip install pyvi

[INFO] Running in Google Colab, installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu1

In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "/content/luatgt2.pdf"
print(f"File {pdf_path} exists.")

File /content/luatgt2.pdf exists.


In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number +1,  # adjust page numbers since our PDF starts on page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
print(f"Total pages: {len(pages_and_texts)}")
pages_and_texts

0it [00:00, ?it/s]

Total pages: 109


[{'page_number': 1,
  'page_char_count': 1628,
  'page_word_count': 320,
  'page_sentence_count_raw': 4,
  'page_token_count': 407.0,
  'text': 'CHÍNH PHỦ -------- CỘNG HÒA XÃ HỘI CHỦNGHĨA VIỆT NAM Độc lập - Tựdo - Hạnh phúc --------------- Số: 168/2024/NĐ-CP Hà Nội, ngày 26 tháng 12 năm 2024 NGHỊĐỊNH QUY ĐỊNH XỬPHẠT VI PHẠM HÀNH CHÍNH VỀTRẬT TỰ, AN TOÀN GIAO THÔNG TRONG LĨNH VỰC GIAO THÔNG ĐƯỜNG BỘ; TRỪĐIỂM, PHỤC HỒI ĐIỂM GIẤY PHÉP LÁI XE Căn cứLuật Tổchức Chính phủngày 19 tháng 6 năm 2015; Luật sửa đổi, bổsung một sốđiều của Luật Tổchức Chính phủvà Luật Tổchức chính quyền địa phương ngày 22 tháng 11 năm 2019; Căn cứLuật Xửlý vi phạm hành chính ngày 20 tháng 6 năm 2012; Luật sửa đổi, bổsung một sốđiều của Luật Xửlý vi phạm hành chính ngày 15 tháng 11 năm 2020; Căn cứLuật Trật tự, an toàn giao thông đường bộngày 27 tháng 6 năm 2024; Theo đềnghịcủa Bộtrưởng BộCông an; Chính phủban hành Nghịđịnh quy định xửphạt vi phạm hành chính vềtrật tự, an toàn giao thông trong lĩnh vực giao thông đư

In [ ]:
import random
random.sample(pages_and_texts, k=3)

[{'page_number': 30,
  'page_char_count': 1739,
  'page_word_count': 352,
  'page_sentence_count_raw': 10,
  'page_token_count': 434.75,
  'text': 'a) Không nhường đường theo quy định, không báo hiệu bằng tay khi chuyển hướng; b) Không chấp hành hiệu lệnh hoặc chỉdẫn của đèn tín hiệu, biển báo hiệu, vạch kẻ đường, trừhành vi vi phạm quy định tại khoản 3 Điều này; c) Không đủdụng cụđựng chất thải của vật nuôi hoặc không dọn sạch chất thải của vật nuôi thải ra đường, vỉa hè; d) Đểvật nuôi đi trên đường bộkhông bảo đảm an toàn cho người, phương tiện đang tham gia giao thông; đ) Đi dàn hàng ngang từ02 xe trởlên; e) Đểvật nuôi kéo xe mà không có người điều khiển; g) Điều khiển xe không có báo hiệu theo quy định. 2. Phạt tiền từ400.000 đồng đến 600.000 đồng đối với một trong các hành vi vi phạm sau đây: a) Không chấp hành hiệu lệnh, hướng dẫn của người điều khiển giao thông hoặc người kiểm soát giao thông; b) Dẫn dắt vật nuôi chạy theo khi đang điều khiển hoặc ngồi trên phương tiện giao thôn

In [ ]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,1628,320,4,407.00,CHÍNH PHỦ -------- CỘNG HÒA XÃ HỘI CHỦNGHĨA VI...
1,2,1895,386,13,473.75,2. Các hành vi vi phạm hành chính trong lĩnh v...
2,3,1743,353,7,435.75,"Điều 3. Hình thức xửphạt vi phạm hành chính, b..."
3,4,2427,489,3,606.75,"h) Buộc tổchức tập huấn, hướng dẫn nghiệp vụ, ..."
4,5,2823,554,10,705.75,"nhận đăng ký xe; giấy chứng nhận, tem kiểm địn..."


In [ ]:
# Get stats
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,109.00,109.00,109.00,109.00,109.00
mean,55.00,2295.78,476.85,7.64,573.94
std,31.61,375.01,77.52,4.11,93.75
min,1.00,1257.00,250.00,1.00,314.25
25%,28.00,2063.00,441.00,5.00,515.75
50%,55.00,2302.00,477.00,7.00,575.50
75%,82.00,2539.00,525.00,10.00,634.75
max,109.00,3138.00,624.00,19.00,784.50


In [ ]:
from spacy.lang.vi import Vietnamese

nlp = Vietnamese(use_pyvi=False)

# Thêm pipeline Sentencizer để tách câu
nlp.add_pipe("sentencizer")

# Văn bản mẫu
doc = nlp("Xin chào! Hôm nay bạn thế nào? Tôi rất vui được gặp bạn.")

# In danh sách câu
for sent in doc.sents:
    print(sent.text)

Xin chào!
Hôm nay bạn thế nào?
Tôi rất vui được gặp bạn.


In [ ]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/109 [00:00<?, ?it/s]

In [ ]:
# Inspect an example
random.sample(pages_and_texts, k=1)

[{'page_number': 16,
  'page_char_count': 2191,
  'page_word_count': 462,
  'page_sentence_count_raw': 5,
  'page_token_count': 547.75,
  'text': 'Điều 7. Xửphạt, trừđiểm giấy phép lái của người điều khiển xe mô tô, xe gắn máy, các loại xe tương tựxe mô tô và các loại xe tương tựxe gắn máy vi phạm quy tắc giao thông đường bộ 1. Phạt tiền từ200.000 đồng đến 400.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Không chấp hành hiệu lệnh, chỉdẫn của biển báo hiệu, vạch kẻđường, trừcác hành vi vi phạm quy định tại điểm b, điểm d, điểm e khoản 2; điểm a, điểm c, điểm d, điểm h khoản 3; điểm a, điểm b, điểm c, điểm d khoản 4; điểm b, điểm d khoản 6; điểm a, điểm b, điểm c khoản 7; điểm a khoản 8; điểm b khoản 9; điểm a khoản 10 Điều này; b) Không có tín hiệu trước khi vượt hoặc có tín hiệu vượt xe nhưng không sửdụng trong suốt quá trình vượt xe; c) Lùi xe mô tô ba bánh không quan sát hai bên, phía sau xe hoặc không có tín hiệu lùi xe; d) Chởngười ngồi t

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,109.00,109.00,109.00,109.00,109.00,109.00
mean,55.00,2295.78,476.85,7.64,573.94,9.67
std,31.61,375.01,77.52,4.11,93.75,5.63
min,1.00,1257.00,250.00,1.00,314.25,1.00
25%,28.00,2063.00,441.00,5.00,515.75,6.00
50%,55.00,2302.00,477.00,7.00,575.50,9.00
75%,82.00,2539.00,525.00,10.00,634.75,12.00
max,109.00,3138.00,624.00,19.00,784.50,28.00


In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/109 [00:00<?, ?it/s]

In [ ]:
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages_and_texts, k=1)

[{'page_number': 75,
  'page_char_count': 2179,
  'page_word_count': 443,
  'page_sentence_count_raw': 15,
  'page_token_count': 544.75,
  'text': '1. Phạt tiền từ100.000 đồng đến 200.000 đồng đối với một trong các hành vi vi phạm sau đây: a) Không chấp hành hướng dẫn của lái xe, nhân viên phục vụtrên xe vềcác quy định bảo đảm trật tự, an toàn giao thông; b) Gây mất trật tựtrên xe. 2. Phạt tiền từ1.000.000 đồng đến 2.000.000 đồng đối với một trong các hành vi vi phạm sau đây: a) Mang hóa chất độc hại, chất dễcháy, nổ, hàng nguy hiểm hoặc hàng cấm lưu thông trên xe chởkhách; b) Đu, bám vào thành xe; đứng, ngồi, nằm trên mui xe, nóc xe, trong khoang chởhành lý; tựý mởcửa xe hoặc có hành vi khác không bảo đảm an toàn khi xe đang chạy. 3. Phạt tiền từ2.000.000 đồng đến 3.000.000 đồng đối với hành vi đe dọa, xâm phạm sức khỏe của người khác đi trên xe, lái xe, nhân viên phục vụtrên xe. 4. Ngoài việc bịphạt tiền, người thực hiện hành vi vi phạm quy định tại điểm a khoản 2 Điều này còn bịáp d

In [ ]:
# Create a DataFrame to get stats
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,109.00,109.00,109.00,109.00,109.00,109.00,109.00
mean,55.00,2295.78,476.85,7.64,573.94,9.67,1.38
std,31.61,375.01,77.52,4.11,93.75,5.63,0.62
min,1.00,1257.00,250.00,1.00,314.25,1.00,1.00
25%,28.00,2063.00,441.00,5.00,515.75,6.00,1.00
50%,55.00,2302.00,477.00,7.00,575.50,9.00,1.00
75%,82.00,2539.00,525.00,10.00,634.75,12.00,2.00
max,109.00,3138.00,624.00,19.00,784.50,28.00,3.00


In [ ]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/109 [00:00<?, ?it/s]

150

In [ ]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 14,
  'sentence_chunk': 'nguyên hiện trường, không trợgiúp người bịnạn, không ởlại hiện trường hoặc không đến trình báo ngay với cơ quan công an, Ủy ban nhân dân nơi gần nhất.9. Phạt tiền từ18.000.000 đồng đến 20.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Điều khiển xe trên đường mà trong máu hoặc hơi thởcó nồng độcồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở; b) Không chấp hành hiệu lệnh của đèn tín hiệu giao thông; c) Không chấp hành hiệu lệnh, hướng dẫn của người điều khiển giao thông hoặc người kiểm soát giao thông; d) Đi ngược chiều của đường một chiều, đi ngược chiều trên đường có biển "Cấm đi ngược chiều”, trừcác hành vi vi phạm quy định tại điểm đ khoản 11 Điều này và các trường hợp xe ưu tiên đang đi làm nhiệm vụkhẩn cấp theo quy định.10. Phạt tiền từ20.000.000 đồng đến 22.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi v

In [ ]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,150.00,150.00,150.00,150.00
mean,54.98,1666.11,344.63,416.53
std,29.26,819.88,171.56,204.97
min,1.00,78.00,16.00,19.50
25%,32.00,955.25,196.25,238.81
50%,55.50,1758.50,374.00,439.62
75%,78.00,2349.25,487.25,587.31
max,109.00,2987.00,624.00,746.75


In [ ]:
filtered_df = df[df["chunk_token_count"] <= min_token_length]
sample_size = min(5, len(filtered_df))  # Đảm bảo không lấy nhiều hơn số phần tử có sẵn

for row in filtered_df.sample(sample_size).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')


NameError: name 'min_token_length' is not defined

In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

NameError: name 'min_token_length' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
sentences_1 = ["Trí tuệ nhân tạo là gì", "Lợi ích của giấc ngủ"]
sentences_2 = ["Trí tuệ nhân tạo là công nghệ giúp máy móc suy nghĩ và học hỏi như con người. Nó hoạt động bằng cách thu thập dữ liệu, nhận diện mẫu và đưa ra quyết định.",
               "Giấc ngủ giúp cơ thể và não bộ nghỉ ngơi, hồi phục năng lượng và cải thiện trí nhớ. Ngủ đủ giấc giúp tinh thần tỉnh táo và làm việc hiệu quả hơn."]
query_embedding = model.encode(sentences_1)
doc_embeddings = model.encode(sentences_2)
similarity = query_embedding @ doc_embeddings.T
print(similarity)
# Kiểm tra lại shape
print(f"Query embedding shape: {query_embedding.shape}")
print(f"Doc embedding shape: {doc_embeddings.shape}")

# Nhân ma trận để tính độ tương đồng
similarity = query_embedding @ doc_embeddings.T
print(similarity)

In [ ]:
%%time

# Send the model to the GPU
embedding_model.to("cuda") # requires a GPU installed, for reference on my local machine, I'm using a NVIDIA RTX 4090

# Create embeddings one by one on the GPU
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

In [ ]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

In [ ]:
import random

import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

In [ ]:
text_chunks_and_embedding_df.head()

In [ ]:
embeddings[0]

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
sentences_1 = ["Trí tuệ nhân tạo là gì", "Lợi ích của giấc ngủ"]
sentences_2 = ["Trí tuệ nhân tạo là công nghệ giúp máy móc suy nghĩ và học hỏi như con người. Nó hoạt động bằng cách thu thập dữ liệu, nhận diện mẫu và đưa ra quyết định.",
               "Giấc ngủ giúp cơ thể và não bộ nghỉ ngơi, hồi phục năng lượng và cải thiện trí nhớ. Ngủ đủ giấc giúp tinh thần tỉnh táo và làm việc hiệu quả hơn."]
query_embedding = model.encode(sentences_1)
doc_embeddings = model.encode(sentences_2)
similarity = query_embedding @ doc_embeddings.T
print(similarity)

[[0.6621206  0.33066636]
 [0.25866616 0.58652884]]


In [ ]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "Nguyên tắc trừ điểm giấy phép lái xe"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)
query_embedding = query_embedding.to("cuda")
embedding_model.to("cuda")
# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: Nguyên tắc trừ điểm giấy phép lái xe


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1024 and 3x147)

In [ ]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
larger_embeddings.to("cuda")
print(f"Embeddings shape: {larger_embeddings.shape}")

# Perform dot product across 168,000 embeddings
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

Embeddings shape: torch.Size([14700, 768])
Time take to get scores on 14700 embeddings: 0.00044 seconds.


In [ ]:
# Define helper function to print wrapped text
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
print(f"Query: '{query}'\n")
print("Results:")
# Loop through zipped together scores and indicies from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    # Print the page number too so we can reference the textbook further (and check the results)
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'Nguyên tắc trừ điểm giấy phép lái xe'

Results:
Score: 0.7549
Text:
Phạt tiền từ10.000.000 đồng đến 15.000.000 đồng đối với cơ sởthửnghiệm, chứng
nhận xe cơ giới, xe máy chuyên dùng, phụtùng xe cơ giới trong sản xuất, lắp ráp
và nhập khẩu thực hiện một trong các hành vi vi phạm sau đây:
Page number: 84


Score: 0.7433
Text:
000 đồng đến 200.000 đồng đối với người được chởtrên xe đạp, xe đạp máy sửdụng ô
(dù).2. Phạt tiền từ200.000 đồng đến 250.000 đồng đối với cá nhân thực hiện một
trong các hành vi vi phạm sau đây: a) Tập trung đông người trái phép, nằm, ngồi
trên đường bộgây cản trởgiao thông; b) Đá bóng, đá cầu, chơi cầu lông hoặc các
hoạt động thểthao khác trái phép trên đường bộ; sửdụng bàn trượt, pa-tanh, các
thiết bịtương tựtrên phần đường xe chạy;
Page number: 30


Score: 0.7401
Text:
000.000 đồng đến 5.000.000 đồng đối với hành vi vi phạm vận chuyển thực phẩm
tươi sống không chấp hành quy định vềan toàn thực phẩm, vệsinh dịch tễ, phòng
dịch và bảo đảm vệsinh môi trường